In [2]:
from ds_util import get_dataset
from perceptron_model import MLPModel
import numpy as np
import torch
import h5py
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from scipy.stats import spearmanr, pearsonr
from tqdm import tqdm

In [3]:
predictions_savepath = '/clusterfs/nilah/oberon/datasets/cs282a/inference/mlp_model_20231128_063541_2.h5'

In [4]:
f = h5py.File('/clusterfs/nilah/oberon/datasets/basenji/embeddings/embeddings.h5','r')

In [5]:
dset = f['embeddings']

In [6]:
trained_model = MLPModel()
trained_model.load_state_dict(torch.load('model_20231128_063541_2'))
trained_model.eval()

MLPModel(
  (layers): Sequential(
    (conv1x1): Conv1d(1536, 500, kernel_size=(1,), stride=(1,))
    (gelu1): GELU(approximate='none')
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (fc1): Linear(in_features=448000, out_features=18, bias=True)
  )
)

In [7]:
with h5py.File(predictions_savepath,'w') as savefile:
    savefile.create_dataset(
        'single_bin',
        shape=(len(dset),1,18),
        chunks=(1,1,18),
        compression='gzip',
        compression_opts=9,
    )
    for i in tqdm(range(len(dset))):
        inputs = torch.Tensor(f['embeddings'][i]).reshape([1,896,1536])
        predictions = trained_model(inputs.transpose(1,2))
        savefile['single_bin'][i,:,:] = predictions.detach()


100%|██████████| 38171/38171 [27:29<00:00, 23.15it/s]  
